## Gold: Dimensão Países

In [0]:
%run "../1 - Setup/Ingestao_bibliotecas_padrao"


In [0]:
from pyspark.sql.functions import col

In [0]:
tabela_silver = spark.table("vitivinicultura.silver_exportacao.exportacao_vinhos")

In [0]:

colunas_anos = [str(ano) for ano in range(1970, 2023)]  

expr = "stack({n}, {pairs}) as (ano, valor)".format(
    n=len(colunas_anos),
    pairs=", ".join([f"'{c}', `{c}`" for c in colunas_anos])
)

df_unpivot = tabela_silver.selectExpr("`País`", expr)


In [0]:
df_unpivot.createOrReplaceTempView("paises")
df = spark.sql("""
    SELECT DISTINCT `País` as PAIS
    FROM paises
    WHERE 'País' IS NOT NULL
""")
df.createOrReplaceTempView("paises_distinct")

In [0]:
df = spark.sql("""
    SELECT ROW_NUMBER() OVER (ORDER BY PAIS ASC) AS ID_PAIS,
    PAIS
    FROM paises_distinct
    WHERE PAIS IS NOT NULL
""")
df.createOrReplaceTempView("paises")

In [0]:
 #Cria tabela gerenciada no Unity Catalog
spark.sql("""
CREATE TABLE IF NOT EXISTS vitivinicultura.gold_cadastros.paises
USING DELTA
""")
# Escrita no formato Delta Lake
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("vitivinicultura.gold_cadastros.paises")

